In [244]:
#command: install required libraries
!pip install mysql-connector-python pandas

In [245]:
!pip install tabulate

# Step 1: Python for Data Processing/Cleaning

In [246]:
#command: import libraries
import pandas as pd
import numpy as np

In [247]:
#command: load dataset
df = pd.read_csv("/content/traffic_stops_with_vehicle_number.csv", low_memory=False)#low_memory=False,Reads the full file first

In [248]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,01-01-2020,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,01-01-2020,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,01-01-2020,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,01-01-2020,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,01-01-2020,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,15-02-2020,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,15-02-2020,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,15-02-2020,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,15-02-2020,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [249]:
#command: remove columns that are completely empty (all NaN or blank)
df = df.replace('', np.nan) #Converts blank cells to proper missing values (NaN)
df = df.dropna(axis=1, how='all') #Remove columns that contain only missing values

In [250]:
# numeric: median; categorical: 'Unknown'
#part of handling missing values for numeric data
if 'driver_age' in df.columns:
    df['driver_age'] = pd.to_numeric(df['driver_age'], errors='coerce') #Converts non-numeric text to NaN
    df['driver_age'] = df['driver_age'].fillna(df['driver_age'].median()) #Fills missing values with the median age

In [251]:
for c in ['driver_gender', 'driver_race', 'violation', 'stop_outcome', 'search_type', 'stop_duration']:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')

In [252]:
for c in ['search_conducted', 'is_arrested', 'drugs_related_stop']:
    if c in df.columns:
        df[c] = df[c].fillna(False).astype(bool) #Replace NaN with False

In [253]:
# Fix date column
if 'stop_date' in df.columns:
    df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce', dayfirst=True)
    df['stop_date'] = df['stop_date'].where(df['stop_date'].notna(), None)
    df.loc[df['stop_date'].notna(), 'stop_date'] = df.loc[df['stop_date'].notna(), 'stop_date'].dt.strftime('%Y-%m-%d')

In [254]:
print("✅ Cleaned data ready")
df.head(10) #default-first 5 rows of df

✅ Cleaned data ready


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
5,2020-01-01,00:05:00,Canada,F,44,73,Other,Drunk Driving,Other,True,Vehicle Search,Ticket,False,16-30 Min,False,MH89ZF7676
6,2020-01-01,00:06:00,USA,F,80,70,White,Signal Violation,Speeding,False,Vehicle Search,Ticket,False,30+ Min,False,KA40AD3153
7,2020-01-01,00:07:00,India,M,72,50,Asian,Drunk Driving,Other,False,Unknown,Arrest,True,30+ Min,True,DL75KZ7835
8,2020-01-01,00:08:00,USA,F,80,30,Hispanic,Other,Seatbelt,True,Frisk,Arrest,False,0-15 Min,True,DL50PO5101
9,2020-01-01,00:09:00,USA,M,43,21,Asian,Signal Violation,Signal,True,Unknown,Warning,True,16-30 Min,False,KA79RD2271


#Step 2: Database Design (SQL)
Connect to MySQL / TiDB Cloud

In [255]:
import mysql.connector

#command: connection details — edit for TiDB or MySQL
config = {
    "host": "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",   # TiDB Cloud endpoint or localhost
    "user": "2zVXFyir6Ee9vfN.root",
    "password": "ONm0NedrUfZ8u7lF",
    "database": "ds",  # database name
    "port": 4000   # TiDB Cloud uses 4000
}#**config means:Unpack the dictionary and pass each key-value pair as a parameter

#command: connect to the database
conn = mysql.connector.connect(**config) #create connection b/w py and sql
cursor = conn.cursor(buffered=True) #cursor-Allows running SQL queries
print("✅ Connected to MySQL / TiDB Cloud!")


✅ Connected to MySQL / TiDB Cloud!


In [256]:
#command: create database and table
cursor.execute("CREATE DATABASE IF NOT EXISTS securecheck_db;")
cursor.execute("USE securecheck_db;")

In [257]:
#command: create database and table
cursor.execute("CREATE DATABASE IF NOT EXISTS securecheck_db;")
cursor.execute("USE securecheck_db;")

create_table_sql = """
CREATE TABLE IF NOT EXISTS traffic_stops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time VARCHAR(16),
    country_name VARCHAR(128),
    driver_gender VARCHAR(16),
    driver_age INT,
    driver_race VARCHAR(64),
    violation VARCHAR(128),
    search_conducted BOOLEAN,
    search_type VARCHAR(64),
    stop_outcome VARCHAR(64),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(32),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(64),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cursor.execute(create_table_sql)
print("✅ Table created successfully")


✅ Table created successfully


Insert Cleaned Data into MySQL

In [258]:
#command: prepare data and insert into MySQL
cols = [
    'stop_date','stop_time','country_name','driver_gender','driver_age',
    'driver_race','violation','search_conducted','search_type',
    'stop_outcome','is_arrested','stop_duration','drugs_related_stop','vehicle_number'
]
cols = [c for c in cols if c in df.columns]
df_insert = df[cols].copy()

In [259]:
#command: replace NaN(pandas cleaning) with None (for MySQL)
df_insert = df_insert.where(pd.notnull(df_insert), None)

In [260]:
#command: convert dataframe rows to list of tuples #sending your cleaned data from pandas → MySQL.
records = [tuple(x) for x in df_insert.to_numpy()]
#| Part                      | Meaning                                   |
#| ------------------------- | ----------------------------------------- |
#| `df_insert.to_numpy()`    | Converts DataFrame → array of rows        |
#| `tuple(x)`                | Converts each row → tuple                 |
#| `[tuple(x) for x in ...]` | Creates a list of all row tuples          |
#| `records`                 | Final list of tuples to insert into MySQL |


In [261]:
#command: build insert query dynamically
placeholders = ', '.join(['%s'] * len(cols)) #%s -prevents No messy string concatenation
col_names = ', '.join(cols)
insert_query = f"INSERT INTO traffic_stops ({col_names}) VALUES ({placeholders})"

In [262]:
#command: insert into MySQL
cursor.executemany(insert_query, records)  # insert entire rows as example
conn.commit()
print(f"✅ Inserted {cursor.rowcount} records successfully!")

✅ Inserted 65538 records successfully!


In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  datetime64[ns]
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age_raw      65538 non-null  int64         
 5   driver_age          65538 non-null  int64         
 6   driver_race         65538 non-null  object        
 7   violation_raw       65538 non-null  object        
 8   violation           65538 non-null  object        
 9   search_conducted    65538 non-null  bool          
 10  search_type         65538 non-null  object        
 11  stop_outcome        65538 non-null  object        
 12  is_arrested         65538 non-null  bool          
 13  stop_duration       65538 non-null  object    

#SQL Analytics Queries

(Medium level):
🚗 Vehicle-Based

In [264]:
#1.What are the top 10 vehicle_Number involved in drug-related stops?
query = """
SELECT vehicle_number, COUNT(*) AS total
FROM traffic_stops
WHERE drugs_related_stop = 1
GROUP BY vehicle_number
ORDER BY total DESC
LIMIT 10;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql')) #keywords arguments and psql postgray sql

+------------------+---------+
| vehicle_number   |   total |
|------------------+---------|
| RJ83PZ4441       |      41 |
| TN41KH6183       |      34 |
| DL88UQ8583       |      34 |
| TN50GM4181       |      34 |
| TN93GT7513       |      34 |
| GJ97NV7324       |      34 |
| DL25IB2226       |      34 |
| DL66CO8373       |      34 |
| TN67IV4899       |      34 |
| DL86TM2706       |      34 |
+------------------+---------+


In [265]:
#2.Which vehicles were most frequently searched?
query = """
SELECT vehicle_number, COUNT(*) AS total
FROM traffic_stops
WHERE search_conducted = 1
GROUP BY vehicle_number
ORDER BY total DESC
LIMIT 5;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+------------------+---------+
| vehicle_number   |   total |
|------------------+---------|
| KA66KF7546       |      34 |
| WB15XR3620       |      34 |
| MH51GC9386       |      34 |
| KA26JU7792       |      34 |
| DL60TN8277       |      34 |
+------------------+---------+


🧍 Demographic-Based

In [266]:
#3.Which driver age group had the highest arrest rate?
query = """
SELECT driver_age, COUNT(*) AS total
FROM traffic_stops
WHERE is_arrested = 1
GROUP BY driver_age
ORDER BY total DESC
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+--------------+---------+
|   driver_age |   total |
|--------------+---------|
|           28 |   18666 |
|           26 |   17832 |
|           34 |   17824 |
|           60 |   17770 |
|           62 |   17688 |
|           38 |   17678 |
|           63 |   17670 |
|           21 |   17660 |
|           53 |   17622 |
|           56 |   17594 |
|           19 |   17388 |
|           37 |   17378 |
|           22 |   17368 |
|           76 |   17332 |
|           48 |   17320 |
|           25 |   17268 |
|           59 |   17208 |
|           40 |   17200 |
|           75 |   17162 |
|           44 |   17112 |
|           67 |   17106 |
|           39 |   17028 |
|           18 |   17012 |
|           68 |   17012 |
|           45 |   16998 |
|           57 |   16926 |
|           66 |   16894 |
|           61 |   16886 |
|           41 |   16884 |
|           77 |   16878 |
|           55 |   16858 |
|           27 |   16848 |
|           35 |   16840 |
|           74 |   16806 |
|

In [267]:
#4.What is the gender distribution of drivers stopped in each country?
query = """
SELECT country_name, driver_gender, COUNT(*) AS total
FROM traffic_stops
GROUP BY country_name, driver_gender
ORDER BY country_name, driver_gender;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+----------------+-----------------+---------+
| country_name   | driver_gender   |   total |
|----------------+-----------------+---------|
| Canada         | F               |  353618 |
| Canada         | M               |  350782 |
| India          | F               |  355032 |
| India          | M               |  352270 |
| India          | male            |       7 |
| USA            | F               |  348590 |
| USA            | M               |  346924 |
+----------------+-----------------+---------+


In [268]:
#5.Which race and gender combination has the highest search rate?
query = """
SELECT driver_race, driver_gender, COUNT(*) AS total
FROM traffic_stops
WHERE search_conducted = 1
GROUP BY driver_race, driver_gender
ORDER BY total DESC
LIMIT 5;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+---------------+-----------------+---------+
| driver_race   | driver_gender   |   total |
|---------------+-----------------+---------|
| Asian         | F               |  109698 |
| Hispanic      | F               |  106978 |
| Other         | F               |  105848 |
| Black         | M               |  104950 |
| White         | M               |  104900 |
+---------------+-----------------+---------+


🕒 Time & Duration Based

In [269]:
#6.What time of day sees the most traffic stops?
query = """
SELECT HOUR(stop_time) AS hour_of_day, COUNT(*) AS total
FROM traffic_stops
GROUP BY hour_of_day
ORDER BY total DESC
LIMIT 1;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+---------------+---------+
|   hour_of_day |   total |
|---------------+---------|
|             1 |   88807 |
+---------------+---------+


In [270]:
#7.What is the average stop duration for different violations?
query = """
SELECT violation, AVG(stop_duration) AS avg_duration_minutes
FROM traffic_stops
GROUP BY violation;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+------------------------+
| violation   |   avg_duration_minutes |
|-------------+------------------------|
| Other       |                15.7427 |
| Seatbelt    |                15.335  |
| Speeding    |                15.2895 |
| Signal      |                15.2039 |
|             |                16      |
| DUI         |                15.2329 |
+-------------+------------------------+


In [271]:
#8. Are stops during the night more likely to lead to arrests?
query = """
SELECT is_arrested, COUNT(*) AS total
FROM traffic_stops
WHERE HOUR(stop_time) >= 22 OR HOUR(stop_time) <= 6
GROUP BY is_arrested;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+---------------+---------+
|   is_arrested |   total |
|---------------+---------|
|             0 |  400478 |
|             1 |  394645 |
|               |       4 |
+---------------+---------+


⚖️ Violation-Based

In [272]:
#9.Which violations are most associated with searches or arrests?
query = """
SELECT violation, COUNT(*) AS total
FROM traffic_stops
WHERE search_conducted = 1 OR is_arrested = 1
GROUP BY violation
ORDER BY total DESC;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------+
| violation   |   total |
|-------------+---------|
| Speeding    |  317260 |
| DUI         |  317234 |
| Other       |  316948 |
| Signal      |  314250 |
| Seatbelt    |  314242 |
|             |       3 |
+-------------+---------+


In [273]:
#10.Which violations are most common among younger drivers (<25)?
query = """
SELECT violation, COUNT(*) AS total
FROM traffic_stops
WHERE driver_age < 25
GROUP BY violation
ORDER BY total DESC;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------+
| violation   |   total |
|-------------+---------|
| Speeding    |   47460 |
| Signal      |   45858 |
| Other       |   45720 |
| Seatbelt    |   45666 |
| DUI         |   43702 |
+-------------+---------+


In [274]:
#11.Is there a violation that rarely results in search or arrest?
query = """
SELECT violation, COUNT(*) AS total
FROM traffic_stops
WHERE search_conducted = 0 AND is_arrested = 0
GROUP BY violation
ORDER BY total ASC;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------+
| violation   |   total |
|-------------+---------|
| DUI         |  103194 |
| Seatbelt    |  103976 |
| Speeding    |  105522 |
| Other       |  107254 |
| Signal      |  107336 |
+-------------+---------+


🌍 Location-Based

In [275]:
#12.Which countries report the highest rate of drug-related stops?
query = """
SELECT country_name, COUNT(*) AS total
FROM traffic_stops
WHERE drugs_related_stop = 1
GROUP BY country_name
ORDER BY total DESC;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+----------------+---------+
| country_name   |   total |
|----------------+---------|
| Canada         |  352832 |
| USA            |  350342 |
| India          |  350307 |
+----------------+---------+


In [276]:
#13.What is the arrest rate by country and violation?
query = """
SELECT country_name, violation, COUNT(*) AS total
FROM traffic_stops
WHERE is_arrested = 1
GROUP BY country_name, violation
ORDER BY country_name, violation;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+----------------+-------------+---------+
| country_name   | violation   |   total |
|----------------+-------------+---------|
| Canada         | DUI         |   72534 |
| Canada         | Other       |   70672 |
| Canada         | Seatbelt    |   71724 |
| Canada         | Signal      |   68418 |
| Canada         | Speeding    |   70246 |
| India          |             |       3 |
| India          | DUI         |   72238 |
| India          | Other       |   70244 |
| India          | Seatbelt    |   70648 |
| India          | Signal      |   72358 |
| India          | Speeding    |   71114 |
| USA            | DUI         |   67750 |
| USA            | Other       |   68598 |
| USA            | Seatbelt    |   68952 |
| USA            | Signal      |   69784 |
| USA            | Speeding    |   70800 |
+----------------+-------------+---------+


In [277]:
#14.Which country has the most stops with search conducted?
query = """
SELECT country_name, COUNT(*) AS total
FROM traffic_stops
WHERE search_conducted = 1
GROUP BY country_name
ORDER BY total DESC;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+----------------+---------+
| country_name   |   total |
|----------------+---------|
| Canada         |  354248 |
| India          |  352384 |
| USA            |  344758 |
+----------------+---------+


(Complex):

In [278]:
#1.Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)
query="""SELECT
    t.country_name,
    t.year,
    t.total_stops,
    t.total_arrests,
    ROUND(t.total_arrests / t.total_stops * 100, 2) AS arrest_rate_percent,
    SUM(t.total_stops) OVER (PARTITION BY t.country_name ORDER BY t.year) AS cumulative_stops,
    SUM(t.total_arrests) OVER (PARTITION BY t.country_name ORDER BY t.year) AS cumulative_arrests
FROM (
    SELECT
        country_name,
        YEAR(stop_date) AS year,
        COUNT(*) AS total_stops,
        SUM(is_arrested) AS total_arrests
    FROM traffic_stops
    WHERE stop_date IS NOT NULL
    GROUP BY country_name, YEAR(stop_date)
) AS t
ORDER BY t.country_name, t.year;""" #Inner subquery (t)

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))


+----------------+--------+---------------+-----------------+-----------------------+--------------------+----------------------+
| country_name   |   year |   total_stops |   total_arrests |   arrest_rate_percent |   cumulative_stops |   cumulative_arrests |
|----------------+--------+---------------+-----------------+-----------------------+--------------------+----------------------|
| Canada         |   2020 |        704400 |          353594 |                 50.2  |             704400 |               353594 |
| India          |   2020 |        707309 |          356605 |                 50.42 |             707309 |               356605 |
| USA            |   2020 |        695514 |          345884 |                 49.73 |             695514 |               345884 |
+----------------+--------+---------------+-----------------+-----------------------+--------------------+----------------------+


In [279]:
#2.Driver Violation Trends Based on Age and Race (Join with Subquery)
query="""SELECT
    v.age_group,
    v.driver_race,
    v.violation,
    v.violation_count,
    t.total_stops,
    ROUND(v.violation_count / t.total_stops * 100, 2) AS violation_percent
FROM
(
    SELECT
        CASE
            WHEN driver_age < 18 THEN 'Under 18'
            WHEN driver_age BETWEEN 18 AND 24 THEN '18-24'
            WHEN driver_age BETWEEN 25 AND 34 THEN '25-34'
            WHEN driver_age BETWEEN 35 AND 49 THEN '35-49'
            ELSE '50+'
        END AS age_group,
        driver_race,
        violation,
        COUNT(*) AS violation_count
    FROM traffic_stops
    WHERE stop_date IS NOT NULL
    GROUP BY age_group, driver_race, violation
) AS v
JOIN (
    SELECT
        CASE
            WHEN driver_age < 18 THEN 'Under 18'
            WHEN driver_age BETWEEN 18 AND 24 THEN '18-24'
            WHEN driver_age BETWEEN 25 AND 34 THEN '25-34'
            WHEN driver_age BETWEEN 35 AND 49 THEN '35-49'
            ELSE '50+'
        END AS age_group,
        driver_race,
        COUNT(*) AS total_stops
    FROM traffic_stops
    WHERE stop_date IS NOT NULL
    GROUP BY age_group, driver_race
) AS t
ON v.age_group = t.age_group AND v.driver_race = t.driver_race
ORDER BY v.age_group, v.driver_race, v.violation_count DESC;
"""




cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------------+-------------+-------------------+---------------+---------------------+
| age_group   | driver_race   | violation   |   violation_count |   total_stops |   violation_percent |
|-------------+---------------+-------------+-------------------+---------------+---------------------|
| 18-24       | Asian         | Speeding    |              9748 |         44466 |               21.92 |
| 18-24       | Asian         | Seatbelt    |              9474 |         44466 |               21.31 |
| 18-24       | Asian         | Other       |              8652 |         44466 |               19.46 |
| 18-24       | Asian         | DUI         |              8400 |         44466 |               18.89 |
| 18-24       | Asian         | Signal      |              8192 |         44466 |               18.42 |
| 18-24       | Black         | Speeding    |             10374 |         46590 |               22.27 |
| 18-24       | Black         | Signal      |             10038 

In [280]:
#3.Time Period Analysis of Stops (Joining with Date Functions)
#, Number of Stops by Year,Month, Hour of the Day
query="""SELECT
    YEAR(stop_date) AS year,
    MONTH(stop_date) AS month,
    HOUR(stop_time) AS hour,
    COUNT(*) AS number_of_stops
FROM traffic_stops
WHERE stop_date IS NOT NULL
GROUP BY year, month, hour
ORDER BY year, month, hour;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+--------+---------+--------+-------------------+
|   year |   month |   hour |   number_of_stops |
|--------+---------+--------+-------------------|
|   2020 |       1 |      0 |             59640 |
|   2020 |       1 |      1 |             59647 |
|   2020 |       1 |      2 |             59640 |
|   2020 |       1 |      3 |             59640 |
|   2020 |       1 |      4 |             59640 |
|   2020 |       1 |      5 |             59640 |
|   2020 |       1 |      6 |             59640 |
|   2020 |       1 |      7 |             59640 |
|   2020 |       1 |      8 |             59640 |
|   2020 |       1 |      9 |             59640 |
|   2020 |       1 |     10 |             59640 |
|   2020 |       1 |     11 |             59640 |
|   2020 |       1 |     12 |             59640 |
|   2020 |       1 |     13 |             59640 |
|   2020 |       1 |     14 |             59640 |
|   2020 |       1 |     15 |             59640 |
|   2020 |       1 |     16 |             59640 |


In [281]:
#4.Violations with High Search and Arrest Rates (Window Function)
query="""
SELECT
    violation,
    COUNT(*) AS total_stops,
    SUM(search_conducted) AS total_searches,
    SUM(is_arrested) AS total_arrests,
    ROUND(SUM(search_conducted) / COUNT(*) * 100, 2) AS search_rate_percent,
    ROUND(SUM(is_arrested) / COUNT(*) * 100, 2) AS arrest_rate_percent,
    RANK() OVER (ORDER BY ROUND(SUM(search_conducted) / COUNT(*) * 100, 2) DESC) AS search_rate_rank,
    RANK() OVER (ORDER BY ROUND(SUM(is_arrested) / COUNT(*) * 100, 2) DESC) AS arrest_rate_rank
FROM traffic_stops
GROUP BY violation
ORDER BY search_rate_rank, arrest_rate_rank;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------------+------------------+-----------------+-----------------------+-----------------------+--------------------+--------------------+
| violation   |   total_stops |   total_searches |   total_arrests |   search_rate_percent |   arrest_rate_percent |   search_rate_rank |   arrest_rate_rank |
|-------------+---------------+------------------+-----------------+-----------------------+-----------------------+--------------------+--------------------|
| Seatbelt    |        418218 |           210994 |          211324 |                 50.45 |                 50.53 |                  1 |                  2 |
| Speeding    |        422782 |           211288 |          212160 |                 49.98 |                 50.18 |                  2 |                  3 |
| Other       |        424202 |           211492 |          209514 |                 49.86 |                 49.39 |                  3 |                  5 |
| Signal      |        421586 |           2092

In [282]:
#5.Driver Demographics by Country (Age, Gender, and Race)
query="""
SELECT
    country_name,
    driver_age,
    driver_gender,
    driver_race,
    COUNT(*) AS total_stops
FROM traffic_stops
GROUP BY country_name, driver_age, driver_gender, driver_race
ORDER BY country_name, driver_age, driver_gender, driver_race;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+----------------+--------------+-----------------+---------------+---------------+
| country_name   |   driver_age | driver_gender   | driver_race   |   total_stops |
|----------------+--------------+-----------------+---------------+---------------|
| Canada         |           18 | F               | Asian         |           804 |
| Canada         |           18 | F               | Black         |          1126 |
| Canada         |           18 | F               | Hispanic      |           864 |
| Canada         |           18 | F               | Other         |          1126 |
| Canada         |           18 | F               | White         |          1254 |
| Canada         |           18 | M               | Asian         |           808 |
| Canada         |           18 | M               | Black         |          1186 |
| Canada         |           18 | M               | Hispanic      |           930 |
| Canada         |           18 | M               | Other         |         

In [283]:
#6.Top 5 Violations with Highest Arrest Rates
query="""
SELECT violation, COUNT(*) AS total_stops,
  SUM(is_arrested) AS total_arrests,
  ROUND(SUM(is_arrested) / COUNT(*) * 100, 2) AS arrest_rate_percent
FROM traffic_stops
GROUP BY violation
ORDER BY arrest_rate_percent DESC
LIMIT 5;
"""

cursor.execute(query)
out=cursor.fetchall()
from tabulate import tabulate  #predefined module and submodule
print(tabulate(out,headers=[i[0] for i in cursor.description],  tablefmt='psql'))

+-------------+---------------+-----------------+-----------------------+
| violation   |   total_stops |   total_arrests |   arrest_rate_percent |
|-------------+---------------+-----------------+-----------------------|
| DUI         |        420428 |          212522 |                 50.55 |
| Seatbelt    |        418218 |          211324 |                 50.53 |
| Speeding    |        422782 |          212160 |                 50.18 |
| Signal      |        421586 |          210560 |                 49.94 |
| Other       |        424202 |          209514 |                 49.39 |
+-------------+---------------+-----------------+-----------------------+


#Step 3: Streamlit Dashboard

In [284]:
#command: install streamlit and ngrok
!pip install streamlit pyngrok


In [286]:
#command: create a simple Streamlit app
%%writefile app.py
import streamlit as st
import pandas as pd
import mysql.connector

# ---------- PAGE SETTINGS ----------
st.set_page_config(page_title="Police Log Dashboard", layout="wide")

st.title("🚔 Add New Police Log & Predict Outcome and Violation")

# ---------- CONNECT TO DATABASE ----------
def get_connection():
    return mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",    # example: "gateway01.ap-southeast.tidbcloud.com"
        user="2zVXFyir6Ee9vfN.root",
        password="ONm0NedrUfZ8u7lF",
        database="securecheck_db",
        port=4000            # (for TiDB Cloud)
    )

# ---------- FORM FOR NEW LOG ENTRY ----------
with st.form("log_form"):
    stop_date = st.date_input("Stop Date")
    stop_time = st.time_input("Stop Time")
    country_name = st.text_input("Country Name")
    driver_gender = st.selectbox("Driver Gender", ["male", "female"])
    driver_age = st.number_input("Driver Age", min_value=16, max_value=100)
    driver_race = st.text_input("Driver Race")
    search_conducted = st.selectbox("Was a Search Conducted?", [0, 1])
    search_type = st.text_input("Search Type")
    drugs_related_stop = st.selectbox("Was it Drug Related?", [0, 1])
    stop_duration = st.selectbox("Stop Duration",["0-15 Min","16-30 Min","30+ Min"])
    vehicle_number = st.text_input("Vehicle Number")
      #command: Create a submit button
    submit = st.form_submit_button("Predict Stop Outcome & Violation")

if submit:#while submit,predict and display violation and stop_outcome for this entry

  # ✅ Insert data into MySQL table
    conn = get_connection()
    cursor = conn.cursor(dictionary=True,buffered=True)
    cursor.execute("""
        INSERT INTO traffic_stops (
            stop_date, stop_time, country_name, driver_gender, driver_age,
            driver_race, search_conducted, search_type,
            drugs_related_stop, stop_duration, vehicle_number
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (stop_date, stop_time, country_name, driver_gender, driver_age,
          driver_race, search_conducted, search_type,
          drugs_related_stop, stop_duration, vehicle_number))
    conn.commit()
    cursor.close()
    conn.close()

    # ✅ Show success message
    st.success(f"New entry added for {vehicle_number}")

    # ✅ Predict stop outcome and violation
    conn = get_connection()
    cursor = conn.cursor(dictionary=True,buffered=True)
    #display violation and stop_outcome
    cursor.execute("""
        SELECT violation, stop_outcome
        FROM traffic_stops
        WHERE vehicle_number = %s
    """, (vehicle_number,))
    result = cursor.fetchone()
    violation, stop_outcome = result.values()
    st.write(f"Predicted Violation: {violation}")
    st.write(f"Predicted Stop Outcome: {stop_outcome}")

    cursor.close()
    conn.close()

# ---------- ADVANCED INSIGHTS ----------
st.header("🧠 Advanced Insights")

query_options = [
    "1.top 10 vehicle_Number involved in drug-related stops",
    "2.vehicles were most frequently searched",
    "3.driver age group had the highest arrest rate",
    "4.the gender distribution of drivers stopped in each country",
    "5.race and gender combination has the highest search rate",
    "6.time of day sees the most traffic stops",
    "7.the average stop duration for different violations",
    "8.stops during the night more likely to lead to arrests",
    "9.violations are most associated with searches or arrests",
    "10.violations are most common among younger drivers (<25)",
    "11.there a violation that rarely results in search or arrest",
    "12.countries report the highest rate of drug-related stops",
    "13.the arrest rate by country and violation",
    "14.country has the most stops with search conducted",
]
selected = st.selectbox("Select a Query to Run", query_options)

if st.button("Run Query"):
    conn = get_connection()
    cursor = conn.cursor(dictionary=True,buffered=True)

    if selected == "1.top 10 vehicle_Number involved in drug-related stops":
        query = """
        SELECT vehicle_number, COUNT(*) AS total
        FROM traffic_stops
        WHERE drugs_related_stop = 1
        GROUP BY vehicle_number
        ORDER BY total DESC
        LIMIT 10;
        """
    elif selected == "2.vehicles were most frequently searched":
        query = """
        SELECT vehicle_number, COUNT(*) AS total
        FROM traffic_stops
        WHERE search_conducted = 1
        GROUP BY vehicle_number
        ORDER BY total DESC
        LIMIT 5
        """
    elif selected == "3.driver age group had the highest arrest rate":
        query = """
        SELECT driver_age, COUNT(*) AS total
        FROM traffic_stops
        WHERE is_arrested = 1
        GROUP BY driver_age
        ORDER BY total DESC
        """
    elif selected == "4.the gender distribution of drivers stopped in each country":
        query = """
        SELECT country_name, driver_gender, COUNT(*) AS total
        FROM traffic_stops
        GROUP BY country_name, driver_gender
        ORDER BY country_name, driver_gender
        """
    elif selected == "5.race and gender combination has the highest search rate":
        query = """
        SELECT driver_race, driver_gender, COUNT(*) AS total
        FROM traffic_stops
        WHERE search_conducted = 1
        GROUP BY driver_race, driver_gender
        ORDER BY total DESC
        LIMIT 5;
        """
    elif selected == "6.time of day sees the most traffic stops":
        query = """
        SELECT HOUR(stop_time) AS hour_of_day, COUNT(*) AS total
        FROM traffic_stops
        GROUP BY hour_of_day
        ORDER BY total DESC
        LIMIT 1;
        """
    elif selected == "7.the average stop duration for different violations":
        query = """
        SELECT violation, AVG(stop_duration) AS avg_duration_minutes
        FROM traffic_stops
        GROUP BY violation;
        """
    elif selected == "8.stops during the night more likely to lead to arrests":
        query = """
        SELECT is_arrested, COUNT(*) AS total
        FROM traffic_stops
        WHERE HOUR(stop_time) >= 22 OR HOUR(stop_time) <= 6
        GROUP BY is_arrested;
        """
    elif selected == "9.violations are most associated with searches or arrests":
        query = """
        SELECT violation, COUNT(*) AS total
        FROM traffic_stops
        WHERE search_conducted = 1 OR is_arrested = 1
        GROUP BY violation
        ORDER BY total DESC;
        """
    elif selected == "10.violations are most common among younger drivers (<25)":
        query = """
        SELECT violation, COUNT(*) AS total
        FROM traffic_stops
        WHERE driver_age < 25
        GROUP BY violation
        ORDER BY total DESC;
        """
    elif selected == "11.there a violation that rarely results in search or arrest":
        query = """
        SELECT violation, COUNT(*) AS total
        FROM traffic_stops
        WHERE search_conducted = 0 AND is_arrested = 0
        GROUP BY violation
        ORDER BY total DESC;
        """
    elif selected == "12.countries report the highest rate of drug-related stops":
        query = """
        SELECT country_name, COUNT(*) AS total
        FROM traffic_stops
        WHERE drugs_related_stop = 1
        GROUP BY country_name
        ORDER BY total DESC;
        """
    elif selected == "13.the arrest rate by country and violation":
        query = """
        SELECT country_name, violation, COUNT(*) AS total
        FROM traffic_stops
        WHERE is_arrested = 1
        GROUP BY country_name, violation
        ORDER BY country_name, violation;
        """
    elif selected == "14.country has the most stops with search conducted":
        query = """
        SELECT country_name, COUNT(*) AS total
        FROM traffic_stops
        WHERE search_conducted = 1
        GROUP BY country_name
        ORDER BY total DESC;
        """
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data)
    st.dataframe(df)

    cursor.close()
    conn.close()


Overwriting app.py


In [79]:
!pip install pyngrok
from pyngrok import ngrok


In [289]:
# 🧹 Step 1: Clean up old installations and config
!pip uninstall -y pyngrok
!rm -rf /root/.ngrok2
!pip install pyngrok --quiet

# 🧠 Step 2: Import and add your valid authtoken
from pyngrok import ngrok
!ngrok config add-authtoken "34yRMi4zjk8EouYfzRqonOioZtm_3sCnf2k5ZYEVTsKUA7uUk"

# 🚀 Step 3: Run Streamlit + open tunnel correctly
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(f"✅ Streamlit App Running at: {public_url.public_url}")


Found existing installation: pyngrok 7.4.1
Uninstalling pyngrok-7.4.1:
  Successfully uninstalled pyngrok-7.4.1
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Streamlit App Running at: https://subpopular-predominant-everette.ngrok-free.dev


In [288]:
from pyngrok import ngrok

try:
    # Get a list of all active tunnels
    tunnels = ngrok.get_tunnels()

    if tunnels:
        print("Active tunnels:")
        for tunnel in tunnels:
            print(f"- {tunnel.public_url}")
            # Stop each tunnel individually
            ngrok.disconnect(tunnel.public_url)
        print("Attempted to disconnect active ngrok tunnels.")
    else:
        print("No active ngrok tunnels found started by this session.")

except Exception as e:
    print(f"An error occurred while trying to list or disconnect tunnels: {e}")
    print("You may need to manually stop ngrok processes or check your ngrok dashboard.")

Active tunnels:
- https://subpopular-predominant-everette.ngrok-free.dev
Attempted to disconnect active ngrok tunnels.
